# Overview
*BeamNG.tech / BeamNG.drive*

This tutorial shows you how the library interacts with custom mods. To do so we create a generic mod that runs code on the vehicle and game engine side.

In this tutorial we differentiate between mod and lua module:

mod - refers to a collection of files that change the behavior of the game (see the zip file)

lua module - refers to a lua 'library' that has a global name containing a table 


## Setup

These are the usual imports to create a simple scenario. As the point is to show the mod support, nothing is happening in the scenario. The mod only prints some text to the console, so take a look at the console or the log file you can find in the user path. Note that the console only flushes the last line to the log file if another console output line is generated.

In [1]:
import os
import zipfile
from pathlib import Path

from beamngpy import BeamNGpy, Scenario, Vehicle, set_up_simple_logging

In [2]:
USERPATH = fr'{os.getenv("LOCALAPPDATA")}/BeamNG.drive'
print(USERPATH)

set_up_simple_logging()
beamng = BeamNGpy('localhost', 64256, home='C:\\Users\\Tymon\\Documents\\BeamNG.tech.v0.30.6.0', user=USERPATH)

scenario = Scenario('smallgrid', 'On how to use custom mods')

etk = Vehicle('ego_vehicle', model='etk800', license='AI', extensions=['vehicleEngineCode'])
scenario.add_vehicle(etk, pos=(0, 0, 0), rot_quat=(0, 0, 1, 0))

pickup = Vehicle('some_vehicle', model='pickup', license='AI', extensions=['vehicleEngineCode'])
scenario.add_vehicle(pickup, pos=(5, 0, 0), rot_quat=(0, 0, 1, 0))

2024-03-03 13:59:04,732 |INFO     |beamngpy                      |Started BeamNGpy logging.


C:\Users\Tymon\AppData\Local/BeamNG.drive


## Creating a Simple Mod

Mods can be installed by creating a zip directory in the userfolder, with default location of `%localappdata%/BeamNG.tech/<VERSION>/mods`. They have to recreate the exact same file structure as you find in the game directory in `BeamNG.tech.vX.X.X.X/lua`.

What happens here is that the two lua files in the BeamNGpy directory are zipped into a mod.

In [3]:
userpath = Path(USERPATH, '0.30')

# setting up mod
myModPath = userpath / 'mods' / 'genericResearchMod.zip' 
myModPath.parent.mkdir(parents=True, exist_ok=True)

geCode = 'gameEngineCode.lua'
zipGEpath = str(Path('lua') / 'ge' / 'extensions' / 'util' / geCode)
veCode = 'vehicleEngineCode.lua'
zipVEpath = str(Path('lua') / 'vehicle' / 'extensions' / veCode)
localDir = Path(os.path.abspath('.'))

print(myModPath)
print(localDir)
print(zipGEpath)
print(zipVEpath)

with zipfile.ZipFile(str(myModPath), 'w') as ziph:
    ziph.write(localDir / geCode, arcname=zipGEpath)
    ziph.write(localDir / veCode, arcname=zipVEpath)

C:\Users\Tymon\AppData\Local\BeamNG.drive\0.30\mods\genericResearchMod.zip
c:\Users\Tymon\Documents\BeamNGpy-1.27.1\examples\modInterface
lua\ge\extensions\util\gameEngineCode.lua
lua\vehicle\extensions\vehicleEngineCode.lua


## Testing the mod

To test the mod we start BeamNG.tech and give the python BeamNG class the location of the gameengine mod within the "genericResearchMod.zip/lua/ge/extensions/" directory. This is necessary to register the file as its own lua module within the game.

After registration, it is available as util_gameEngineCode within the game - try typing `dump(util_gameEngineCode)` in the game's command prompt

In [5]:
bng = beamng.open(extensions=['util/gameEngineCode'])
scenario.make(beamng)
bng.scenario.load(scenario)
bng.scenario.start()

2024-03-03 13:59:40,845 |WARNING  |py.warnings                   |c:\Users\Tymon\miniconda3\envs\beamng\Lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=4132, family=2, type=1, proto=0, laddr=('0.0.0.0', 60447)>

2024-03-03 13:59:40,845 |WARNING  |py.warnings                   |c:\Users\Tymon\miniconda3\envs\beamng\Lib\site-packages\beamngpy\connection\connection.py:-1: ResourceWarning: unclosed <socket.socket fd=4132, family=2, type=1, proto=0, laddr=('0.0.0.0', 60447)>

2024-03-03 13:59:40,846 |INFO     |beamngpy.BeamNGpy             |Opening BeamNGpy instance.
2024-03-03 13:59:40,851 |INFO     |beamngpy.BeamNGpy             |Started BeamNG.
2024-03-03 13:59:50,852 |INFO     |beamngpy.BeamNGpy             |Connecting to BeamNG.tech at: (localhost, 64256)
2024-03-03 13:59:50,975 |INFO     |beamngpy.BeamNGpy             |Successfully connected to BeamNG.tech.
2024-03-03 13:59:50,977 |INFO     |beamngpy.BeamNGpy             |BeamNGpy su

## How to Call Lua Module Functions

BeamNG.tech and BeamNGpy communicate via TCP sockets. 
Function parameters are send with the help of python dictionaries that are available as lua tables in the game. 
The mini mod from this tutorial follows the convention the BeamNGpy library established: Every value for the 'type' key helps identifying the appropriate handler, which for 'Foo' is expected to be 'handleFoo'.
The `checkMessages` function in `lua/ge/extensions/tech/techCore.lua` and `lua/vehicle/extensions/tech/techCore.lua` checks whether a corresponding function is locally available and, if not, calls the `onSocketMessage` hook for every extension. 

## Data Transfer between BeamNGpy and BeamNG
Any return value needs to be send with the help of the socket. See here an example on how to do it on the game engine side. The code for the vehicle side is the same, it just needs to be implemented in `lua/vehicle/extensions/tech/techCore.lua`.

In [ ]:
def callFoo(bng):
    '''
    Executes handleFoo defined in gameEngineCode.lua.
    '''
    data = dict(type='Foo', someName='YourName')
    response = bng.connection.send(data)
    response.ack('FooAck')

def callBar(veh):
    '''
    Executes handlebar defined in vehicleEngineCode.lua.
    Here the code is executed in the VM of the etk800.
    '''
    data = dict(type='Bar', text='lorem ipsum...')
    response = veh.connection.send(data)
    response.ack('BarAck')

def dataExchange(bng):
    '''
    Demonstrates how to transfer in game data to the python side.
    '''
    data = dict(type='GetListOfVehicleModels')
    response = bng.connection.send(data).recv('ListOfVehicleModels')
    print('List of spawned vehicle models: ', response['data'])

In [ ]:
callFoo(bng)

BNGError: The request was not handled by BeamNG.tech. This can mean:
- an incompatible version of BeamNG.tech/BeamNGpy
- an internal error in BeamNG.tech
- incorrectly implemented custom command

In [ ]:
callBar(etk)

BNGError: The request was not handled by BeamNG.tech. This can mean:
- an incompatible version of BeamNG.tech/BeamNGpy
- an internal error in BeamNG.tech
- incorrectly implemented custom command

In [ ]:
dataExchange(bng)

BNGError: The request was not handled by BeamNG.tech. This can mean:
- an incompatible version of BeamNG.tech/BeamNGpy
- an internal error in BeamNG.tech
- incorrectly implemented custom command

In [6]:
def getWheelSpeeds(veh):
    data = dict(type='GetWheelSpeeds')
    response = veh.connection.send(data)
    return response.recv()

import time
for _ in range(10):
    print(getWheelSpeeds(etk))
    time.sleep(1.0)

{'FL': 0.0001985326561894025, 'FR': -0.00024019155751480923, 'RL': 0.0003239155953987573, 'RR': -4.050501656638155e-05}
{'FL': -0.00021918694793887717, 'FR': 0.00012448428758579015, 'RL': 0.00010568498071708393, 'RR': 0.00010585274229929317}
{'FL': -0.0001341046413751752, 'FR': 9.776496634924538e-05, 'RL': -9.381233345182587e-05, 'RR': -6.01950762993573e-06}
{'FL': -0.00040989346806633644, 'FR': 3.832611585046965e-05, 'RL': -0.00010345402833690593, 'RR': -8.183875709696551e-05}
{'FL': -0.00033335501771926727, 'FR': -7.979532293809347e-06, 'RL': -0.00015274061112658446, 'RR': 8.169514165807695e-05}
{'FL': -1.2560122123296855e-06, 'FR': -0.00042291890154838764, 'RL': 3.200361253442757e-05, 'RR': -0.0003711077123460736}
{'FL': -0.00024117126729331935, 'FR': -0.0003697670426924729, 'RL': -9.925491957063318e-05, 'RR': -0.00014816228527858645}
{'FL': 7.219912466606667e-05, 'FR': 7.953632570031467e-05, 'RL': 5.4173212858734116e-05, 'RR': 0.00015939221616827727}
{'FL': -2.5206706634692704e-05,

### Don't see anything?

If you have trouble finding the messages in the log file or command prompt, search for 'gameEngineCode' or 'vehicleEngineCode'. These are the log tags of the respective modules.

In [ ]:
bng.close()

2024-02-26 16:51:04,653 |INFO     |beamngpy.BeamNGpy             |Closing BeamNGpy instance.
2024-02-26 16:51:04,655 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle some_vehicle and detached sensors.
2024-02-26 16:51:04,655 |INFO     |beamngpy.Vehicle              |Disconnected from vehicle ego_vehicle and detached sensors.
2024-02-26 16:51:04,656 |INFO     |beamngpy.BeamNGpy             |Terminating BeamNG.tech process.
2024-02-26 16:51:04,798 |WARNING  |py.warnings                   |c:\Users\Tymon\miniconda3\envs\beamng\Lib\subprocess.py:1127: ResourceWarning: subprocess 12616 is still running
  _warn("subprocess %s is still running" % self.pid,

2024-02-26 16:51:04,798 |WARNING  |py.warnings                   |c:\Users\Tymon\miniconda3\envs\beamng\Lib\subprocess.py:1127: ResourceWarning: subprocess 12616 is still running
  _warn("subprocess %s is still running" % self.pid,

2024-02-26 16:51:04,800 |WARNING  |py.warnings                   |c:\Users\Tymon\minicond